In [2]:
import numpy as np

def code1(A, B, C, x0):
    test = -1
    delta = 0.001
    N = 1000
    t = np.linspace(0, 1, N+1)
    h = 1.0 / N
    h2 = h / 2

    # inicialização
    u = np.zeros(N+1)
    x = np.zeros(N+1)
    x[0] = x0
    lam = np.zeros(N+1)

    while test < 0:
        oldu = u.copy()
        oldx = x.copy()
        oldlam = lam.copy()

        # integração do estado (para frente, RK4)
        for i in range(N):
            k1 = -0.5 * x[i]**2 + C * u[i]
            k2 = -0.5 * (x[i] + h2*k1)**2 + C * 0.5 * (u[i] + u[i+1])
            k3 = -0.5 * (x[i] + h2*k2)**2 + C * 0.5 * (u[i] + u[i+1])
            k4 = -0.5 * (x[i] + h*k3)**2 + C * u[i+1]
            x[i+1] = x[i] + (h/6) * (k1 + 2*k2 + 2*k3 + k4)

        # integração do adjunto (para trás, RK4)
        for j in range(N, 0, -1):  # de N até 1
            k1 = -A + lam[j] * x[j]
            k2 = -A + (lam[j] - h2*k1) * 0.5 * (x[j] + x[j-1])
            k3 = -A + (lam[j] - h2*k2) * 0.5 * (x[j] + x[j-1])
            k4 = -A + (lam[j] - h*k3) * x[j-1]
            lam[j-1] = lam[j] - (h/6) * (k1 + 2*k2 + 2*k3 + k4)

        # atualização do controle (condição ótima)
        u1 = (C * lam) / (2 * B)
        u = 0.5 * (u1 + oldu)

        # critério de parada
        temp1 = delta * np.sum(np.abs(u)) - np.sum(np.abs(oldu - u))
        temp2 = delta * np.sum(np.abs(x)) - np.sum(np.abs(oldx - x))
        temp3 = delta * np.sum(np.abs(lam)) - np.sum(np.abs(oldlam - lam))
        test = min(temp1, temp2, temp3)

    return t, x, lam, u


In [3]:
import matplotlib.pyplot as plt

# parâmetros de teste
A, B, C, x0 = 1.0, 1.0, 1.0, 1.0

t, x, lam, u = code1(A, B, C, x0)

# plt.figure(figsize=(10,6))
# plt.plot(t, x, label="Estado x(t)")
# plt.plot(t, lam, label="Adjunto λ(t)")
# plt.plot(t, u, label="Controle u(t)")
# plt.legend()
# plt.grid()
# plt.show()

In [8]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

def system(t, z, A, B, C):
    x, lam = z
    u = (C * lam) / (2 * B)  # condição ótima
    dxdt = -0.5 * x**2 + C * u
    dlamdt = -A + lam * x
    return [dxdt, dlamdt]


In [9]:
# parâmetros
A, B, C, x0, lam0 = 1.0, 1.0, 1.0, 1.0, 0.0
t_span = (0, 1)
t_eval = np.linspace(0, 1, 500)

# integra
sol = solve_ivp(system, t_span, [x0, lam0], args=(A, B, C), t_eval=t_eval)
t = sol.t
x, lam = sol.y
u = (C * lam) / (2 * B)

In [11]:
# plt.figure(figsize=(10,6))
# plt.plot(t, x, label="Estado x(t)")
# plt.plot(t, lam, label="Adjunto λ(t)")
# plt.plot(t, u, label="Controle u(t)")
# plt.xlabel("Tempo")
# plt.ylabel("Valores")
# plt.legend()
# plt.grid()
# plt.show()

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

# Sistema aumentado (x, lam)
def system(t, z, A, B, C):
    x, lam = z
    u = (C * lam) / (2 * B)  # controle ótimo
    dxdt = -0.5 * x**2 + C * u
    dlamdt = -A + lam * x
    return [dxdt, dlamdt]

# função de shooting
def shooting(lam0_guess, A, B, C, x0):
    lam0 = lam0_guess[0]
    sol = solve_ivp(system, (0, 1), [x0, lam0], args=(A, B, C), dense_output=True)
    lam_T = sol.y[1, -1]  # valor de lambda em t=1
    return lam_T  # queremos lam(T)=0


# parâmetros
A, B, C, x0 = 1.0, 1.0, 1.0, 1.0

# chute inicial para lambda(0)
lam0_guess = [0.5]

# ajusta lambda(0) usando fsolve
lam0_sol = fsolve(shooting, lam0_guess, args=(A, B, C, x0))[0]
print("λ(0) ótimo =", lam0_sol)


# integra com lambda(0) encontrado
t_eval = np.linspace(0, 1, 500)
sol = solve_ivp(system, (0, 1), [x0, lam0_sol], args=(A, B, C), t_eval=t_eval)

t = sol.t
x, lam = sol.y
u = (C * lam) / (2 * B)


In [34]:
# plt.figure(figsize=(10,6))
# plt.plot(t, x, label="Estado x(t)")
# plt.plot(t, lam, label="Adjunto λ(t)")
# plt.plot(t, u, label="Controle u(t)")
# plt.xlabel("Tempo")
# plt.ylabel("Valores")
# plt.legend()
# plt.grid()
# plt.show()
